In [21]:
t: ("DSTFF";enlist ",") 0:`$"trade.csv";
q: ("DSTFFFF";enlist ",") 0:`$"quote.csv";
p: ("DSSSITTFF";enlist ",") 0:`$"parent_order.csv";
c: ("SSDSTFF";enlist ",") 0:`$"child_order.csv";
t:update time:?[time>15:00;15:00:00.000;time] from t
q:update time:?[time>15:00;15:00:00.000;time] from q
/t: update time:09:25|time&15:00 from t;

In [15]:
\c 20 200

In [ ]:
bench:{[benchpx;px;side] 10000*side*(benchpx-px)%benchpx};

In [19]:
tca_calc:{[item]
    t1: select from t where date=item`date, sym=item`sym;
    q1: update midpx:0.5*bid+ask from select from q where date = item`date, sym = item`sym;
    c1: select from c where date=item`date, sym=item`sym, parentid=item`orderid;
    
    d: select DV:sum size, open:first price, close:last price from t1;
    d: d,'select spread: avg 10000*(ask-bid)%0.5*ask+bid from q1 where time within (item`starttime;item`endtime);
    d: d,'select arrival:last midpx from q1 where time<=item`starttime;
    d: d,'select ivwap:size wavg price, ivol: sum size from t1 where time within (item`starttime;item`endtime);
    d: d,'select avgpx:size wavg price, sum size from c1;
    
    / calculate moo moc passive aggressive
    c1: update pass: (item`side) * signum ( midpx-price) from aj[`time;c1;select time,midpx from q1];
    c1: update pass:2 from c1 where time<09:30;
    c1: update pass:-2 from c1 where time>14:57;
    d: d,'select moo: (sum size where pass = 2)%sum size, moc: (sum size where pass = -2)%sum size from c1;   
    d: d,'select passive:(sum size where pass=1)%sum size, aggressive:(sum size where pass=-1)%sum size from c1;

   
    d: d,'select pwp5:size wavg price from (update vol5:sums size*0.05 from select from t1 where time>=item`starttime) where vol5 <=item`qty;
    d: d,'select notional:sum price*size from c1;
    
    d: (enlist item), 'd;
    
    d: update arrival:open from d where starttime<09:30; 
    res: select orderid, notional, adv:size%DV, speed:size%ivol, spread, open:bench[open;avgpx;side], arrival:bench[arrival;avgpx;side], ivwap:bench[ivwap;avgpx;side], close:bench[close;avgpx;side], pwp5:bench[pwp5;avgpx;side],moo: moo, moc: moc, passive, aggressive from d;
    res
  };

In [25]:
result: raze tca_calc each p;
ass2_final_res: select OrderID: orderid, Notional:notional, ADV:adv, Speed:speed, Spread:spread,Open:open, Arrival:arrival, iVWAP:ivwap, Close:close, PWP5:pwp5, MOO:moo, MOC:moc, Passive:passive, Aggressive:aggressive from result
al: update OrderID: `All from select Notional:sum Notional,ADV:Notional wavg ADV, Speed:Notional wavg Speed, Spread:Notional wavg Spread, Open:Notional wavg Open,Arrival:Notional wavg Arrival,iVWAP: Notional wavg iVWAP,Close:Notional wavg Close,PWP5:Notional wavg PWP5,MOO:Notional wavg MOO, MOC:Notional wavg MOC,Passive:Notional wavg Passive,Aggressive:Notional wavg Aggressive from ass2_final_res
al: `OrderID xcols al
`ass2_final_res insert (al)
ass2_final_res


,5


OrderID Notional     ADV         Speed      Spread   Open      Arrival   iVWAP     Close     PWP5      MOO         MOC        Passive   Aggressive
--------------------------------------------------------------------------------------------------------------------------------------------------
V001    1.589476e+07 0.008994977 0.01963853 8.604032 -30.88169 -30.88169 -2.530835 121.4737  0.5800275 0.031875    0          0.322875  0.64525   
V002    2.338996e+07 0.007877907 0.01598804 3.977859 8.149419  25.82842  -4.185585 -33.88749 -13.122   0           0          0.328473  0.6668876 
V003    4472776      0.003490736 0.01091937 7.319823 -69.38475 -69.38475 -6.428954 -112.4393 -25.38996 0.09940778  0          0.4826565 0.4179357 
V004    2.849393e+07 0.01815438  0.01815438 4.058475 3.765729  3.765729  -1.660645 -10.10912 4.979707  0           0.03265925 0.4331648 0.5220425 
V005    3.313667e+07 0.02268216  0.02268216 7.154709 -149.2816 -149.2816 17.85896  -117.6897 93.14445  0.007265411 0.0

In [24]:
save `ass2_final_res.csv

`:ass2_final_res.csv
